# Image Recognition

Il progetto verrà gestito con YOLOv8 e Object365 scaricato localmente con appostito script personalizzato.

## ✅ Step 1 – Preparazione
Assicurarsi che la libreria `ultralytics` sia installata

In [ ]:
%pip install --quiet ultralytics

In [2]:
base_dir = "/mnt/d/objects365"

Il dataset scaricato non ha la cartella di test ma solo `train` e `val` che sono per appunto addestramento e validazione. Ho deciso di creare una cartella di test. 

In [ ]:
from data.create_test_samples import create_test_set

create_test_set(base_dir, test_size_ratio=0.1, num_threads=8)

Siccome YOLO ha bisogno di un file di configurazione in formato YAML (Yet Another Markup Language) e siccome il dataset è diviso in patch c'è bisogno di dare le coordinate delle patch a YOLO e per farlo torna comodo un file TXT con tutti i percorsi.

In [ ]:
from data.create_image_lists import create_image_list


# Crea gli elenchi delle immagini per il training, la validazione e il test
threads = 16
create_image_list(base_dir, "train", f"{base_dir}/train_images.txt", num_threads=threads)
create_image_list(base_dir, "val", f"{base_dir}/val_images.txt", num_threads=threads)
create_image_list(base_dir, "test", f"{base_dir}/test_images.txt", num_threads=threads)

Qui un estratto del file YAML:
```yaml
path: /mnt/d/objects365
train: train_images.txt  # File con elenco immagini di training
val: val_images.txt      # File con elenco immagini di validation
test: test_images.txt    # File con elenco immagini di test
```

Siccome mi sono accorto che nelle etichette spesso compare la classe 365 che non è mappata nel file YAML e siccome YOLO non ha la classe `background` per identificare tutto ciò che non è mappabile dal modello, ho deciso di mappare la classe 365 delle etichette come classe `Unknown`.

```yaml
# Classes
names:
  0: Person
  1: Sneakers
  2: Chair
  3: Other Shoes
  ...
  361: Lipstick
  362: Cosmetics Mirror
  363: Curling
  364: Table Tennis
  365: Unknown
```

Il dataset è molto grande, iniziamo con un subsampling che poi tratteremo come sopra ossia creando un TXT con le coordinate delle immagini e delle labels. Qui sotto un estratto:

In [ ]:
from data.subsampling import create_subsample_image_list

# Esegui la creazione del subsample
train_patches = 6
output_dir = f"/mnt/d/objects365_subsample_{train_patches}"  # Percorso per salvare i file di coordinate
create_subsample_image_list(base_dir, output_dir, train_patches=train_patches, val_patches=1)

## ✅ Step 2 – Addestramento del Modello

Il modello di YOLO scelto sarà un *pretrained* su cui verranno impostati parametri nel tentativo di ottimizzare fin da subito.


In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11s.pt")  # load a pretrained model (recommended for training)

100%|██████████| 18.4M/18.4M [00:00<00:00, 68.8MB/s]


## Augmentation Parameters
Gli aumenti dei dati sono tecniche utilizzate per migliorare la robustezza e la generalizzazione del modello. Ecco i parametri configurabili:

- **hsv_h (Hue)**: Regola la tonalità dei colori per rendere il modello invariante alle variazioni di colore.
- **hsv_s (Saturation)**: Modifica la saturazione per simulare diverse condizioni di illuminazione.
- **hsv_v (Brightness)**: Cambia la luminosità per gestire immagini con diverse esposizioni.
- **degrees**: Applica rotazioni casuali per migliorare l'invarianza al punto di vista.
- **translate**: Introduce traslazioni casuali per rendere il modello robusto a oggetti in posizioni diverse.
- **scale**: Ridimensiona gli oggetti per gestire variazioni di dimensioni.
- **shear**: Applica tagli (shearing) per simulare distorsioni geometriche (impostato a 0 per evitare distorsioni estreme).
- **fliplr**: Effettua un flip orizzontale casuale per aumentare la varietà dei dati.
- **mosaic**: Combina più immagini in un'unica immagine per migliorare il rilevamento di piccoli oggetti e il contesto.
- **bgr**: Scambia i canali BGR per gestire diversi formati di immagine.

In [10]:
import os
workers = max(4, os.cpu_count() - 2)
data = "data/objects365_subsample.yaml"
train_params = {
    "data":         data,                    # YAML con path /mnt/e/yolo-fast
    "epochs":       100,                     # numero di epoche
    "patience":     50,                      # early stopping
    "batch":        -1,                      # auto batch size (~60% VRAM)
    "imgsz":        640,                     # dimensione delle immagini
    "device":       0,                       # GPU 0
    "workers":      workers,                 # DataLoader parallelismo
    "cache":        "disk",                  # caching su disco (/mnt/e)
    "save":         True,                    # salva checkpoint e modello
    "save_period":  5,                       # checkpoint ogni 5 epoche
    "project":      "runs/train",            # cartella di uscita
    "name":         "exp_fastcache",         # nome del run
    "exist_ok":     True,                    # sovrascrivi se esiste
    "pretrained":   True,                    # usa pesi pretrained
    "optimizer":    "AdamW",                 # SGD o Adam/AdamW
    "amp":          True,                    # mixed precision
    "rect":         False,                   # rectangular training off
    "multi_scale":  True,                    # multi-scale training on
    "cos_lr":       True,                    # cosine LR scheduler
    "close_mosaic": 10,                      # disabilita mosaic ultime 10 epoche
    "lr0":          0.01,                    # learning rate iniziale
    "lrf":          0.1,                     # final LR = lr0 * lrf
    "momentum":     0.937,                   # momentum / beta1 per Adam
    "weight_decay": 0.0005,                  # regularizzazione L2
    "warmup_epochs":    3.0,                 # epoche warmup LR
    "warmup_momentum":  0.8,                 # momentum warmup
    "warmup_bias_lr":   0.1,                 # bias LR warmup
    "seed":         42,                      # per riproducibilità
    "fraction":     1.0,                     # usa 100% del dataset
    "val":          True,                    # abilita validazione
    "plots":        False                    # salva grafici di training/val
}

In [11]:
# Parametri di data augmentation ottimizzati
augmentation_params = {
    # Colore (HSV)
    "hsv_h":       0.015,   # Default: 0.015 (range 0.0–1.0) – mantiene piccole variazioni di tonalità
    "hsv_s":       0.7,     # Default: 0.7   (range 0.0–1.0) – variazione consistente di saturazione
    "hsv_v":       0.4,     # Default: 0.4   (range 0.0–1.0) – variazione di luminosità medio-alta

    # Geometriche
    "degrees":     5.0,     # Da 0.0 a 180 – rotazioni lievi per preservare orientamenti realistici
    "translate":   0.1,     # Da 0.0 a 1.0   – spostamenti fino al 10% delle dimensioni d’immagine
    "scale":       0.3,     # ≥ 0.0          – riduzione del range (da 0.5 a 0.3) per minor carico computazionale
    "shear":       0.0,     # –180 a +180    – mantenuto a 0 per evitare distorsioni drastiche
    "perspective": 0.0005,  # 0.0–0.001      – inserita una lieve prospettiva per aumentare la robustezza 3D

    # Flip
    "flipud":      0.0,     # 0.0–1.0 – no flip verticale (rischio di immagini “capovolte” poco realistiche)
    "fliplr":      0.5,     # 0.0–1.0 – flip orizzontale con probabilità 50%

    # Canale BGR
    "bgr":         1.0,     # 0.0–1.0 – scambio completo dei canali per insegnare a gestire l’ordine BGR

    # Mosaic / MixUp / CutMix
    "mosaic":      0.5,     # 0.0–1.0 – mosaic ridotto al 50% per bilanciare velocità e varietà
    "mixup":       0.3,     # 0.0–1.0 – mixup moderato per label noise e greater generalization

    # Classification-only (opzionali, ignora se non serve)
    # "erasing":   0.4,     # 0.0–0.9 – random erasing (classification only, lasciare a default se non serve)
    # "auto_augment": "randaugment",  # classification only
}
train_params.update(augmentation_params)


In [ ]:
import json

# Addestra il modello con gli aumenti dei dati
results = model.train(**train_params)
# Crea la cartella 'results' se non esiste
# Percorso per salvare i file
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

model.save(f"{output_dir}/best_model.pt")  # Salva il modello addestrato
# Nome base dei file
filename = f"{output_dir}/training_results"

# Salvataggio in TXT
with open(f"{filename}.txt", "w") as f:
    f.write("Training Results:\n")
    f.write(f"mAP50: {results.get('mAP50', 0.0):.4f}\n")
    f.write(f"mAP50-95: {results.get('mAP50-95', 0.0):.4f}\n")
    f.write(f"Precision: {results.get('precision', 0.0):.4f}\n")
    f.write(f"Recall: {results.get('recall', 0.0):.4f}\n")
    f.write(f"Training Time: {results.get('training_time', 'N/A')}\n")

# Salvataggio in JSON
with open(f"{filename}.json", "w") as f_json:
    json.dump(results, f_json, indent=4)

print(f"✅ Risultati di addestramento salvati in '{filename}.txt' e '{filename}.json'")

New https://pypi.org/project/ultralytics/8.3.123 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.116 🚀 Python-3.12.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=data/objects365_subsample.yaml, epochs=100, time=None, patience=50, batch=-1, imgsz=640, save=True, save_period=5, cache=disk, device=0, workers=18, project=runs/train, name=exp_fastcache, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, s

## ✅ Step 3 – Validazione del Modello

In [ ]:
print(model.info())

In [ ]:
import os
from ultralytics import YOLO

# Path al file di pesi salvati
weights_path = "results/best_model.pt"

# ----------------------------
# 1. Determina se 'model' è già in memoria
# ----------------------------
try:
    # Prova ad accedere a `model`; se non esiste, scatena NameError
    _ = model
    in_ram = True
except NameError:
    in_ram = False

# ----------------------------
# 2. Scegli l'oggetto YOLO per la validazione
# ----------------------------
if in_ram:
    print("✔️ Modello già in RAM: lo uso per la validazione")
    model_to_val = model
elif os.path.exists(weights_path):
    print(f"✔️ Carico modello da disco: {weights_path}")
    model_to_val = YOLO(weights_path)
else:
    raise FileNotFoundError(f"Né modello in RAM né file trovato: {weights_path}")


# Esegui la validazione
val_results = model_to_val.val(
    data=data,
    imgsz=640,
    batch=16,
    device=0,
    save=True
)

# Crea la cartella dei risultati
os.makedirs("results", exist_ok=True)

# Ottieni il dizionario con le metriche
metrics = val_results.results_dict


filename = "results/validation_results_subest_training_4"

# Stampa tutte le chiavi disponibili per debug (facoltativo)
print("Chiavi disponibili in results_dict:")
for k in metrics:
    print(f"- {k}")

# Salvataggio in TXT
with open(f"{filename}.txt", "w") as f:
    f.write("Validation Results:\n")
    f.write(f"mAP50: {metrics.get('mAP50', 0.0):.4f}\n")
    f.write(f"mAP50-95: {metrics.get('mAP50-95', 0.0):.4f}\n")
    f.write(f"Precision: {metrics.get('precision', 0.0):.4f}\n")
    f.write(f"Recall: {metrics.get('recall', 0.0):.4f}\n")
    f.write(f"Inference Speed: {val_results.speed.get('inference', 0.0):.2f} ms/img\n")

# Salvataggio in JSON
with open(f"{filename}.json", "w") as f_json:
    json.dump(metrics, f_json, indent=4)

print(f"✅ Risultati salvati in '{filename}.txt' e '{filename}.json'")
